# Tip Inventory Consolidation

In [1]:
import numpy as np
import random
import time

## Workcell Setup

In [2]:
# === Configuration ===
script_mode = "simulation"  # or "execution"
liquid_handler_choice = "starlet"  # star | ot2 | evo100 | etc.

In [3]:
# For development: auto-reload modules
%load_ext autoreload
%autoreload 2

import logging
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.visualizer.visualizer import Visualizer

# === Liquid handler configuration ===
liquid_handler_config = {
    "star": {
        "deck": ("pylabrobot.resources.hamilton", "STARDeck"),
        "execution": ("pylabrobot.liquid_handling.backends", "STARBackend"),
        "simulation": ("pylabrobot.liquid_handling.backends", "LiquidHandlerChatterboxBackend"),
    },
    "starlet": {
        "deck": ("pylabrobot.resources.hamilton", "STARLetDeck"),
        "execution": ("pylabrobot.liquid_handling.backends", "STARBackend"),
        "simulation": ("pylabrobot.liquid_handling.backends", "LiquidHandlerChatterboxBackend"),
    },
    "vantage": {
        "deck": {
            "module": "pylabrobot.resources.hamilton",
            "class": "VantageDeck",
            "args": { "size": 1.3 }
        },
        "execution": {
            "module": "pylabrobot.liquid_handling.backends",
            "class": "VantageBackend",
        },
        "simulation": {
            "module": "pylabrobot.liquid_handling.backends",
            "class": "LiquidHandlerChatterboxBackend",
        }
    },
    "ot2": {
        "deck": ("pylabrobot.resources.opentrons", "OTDeck"),
        "execution": {
            "module": "pylabrobot.liquid_handling.backends",
            "class": "OpentronsBackend"
        },
        "simulation": {
            "module": "pylabrobot.liquid_handling.backends",
            "class": "LiquidHandlerChatterboxBackend",
            "args": { "num_channels": 1 }
        }
    },
    "evo100": {
        "deck": ("pylabrobot.resources.tecan", "EVO100Deck"),
        "execution": ("pylabrobot.liquid_handling.backends", "EVOBackend"),
        "simulation": ("pylabrobot.liquid_handling.backends", "LiquidHandlerChatterboxBackend"),
    },
    "evo150": {
        "deck": ("pylabrobot.resources.tecan", "EVO150Deck"),
        "execution": ("pylabrobot.liquid_handling.backends", "EVOBackend"),
        "simulation": ("pylabrobot.liquid_handling.backends", "LiquidHandlerChatterboxBackend"),
    },
    "evo200": {
        "deck": ("pylabrobot.resources.tecan", "EVO200Deck"),
        "execution": ("pylabrobot.liquid_handling.backends", "EVOBackend"),
        "simulation": ("pylabrobot.liquid_handling.backends", "LiquidHandlerChatterboxBackend"),
    },
}

# === Liquid handler selection ===
# Define these before running
# liquid_handler_choice = "vantage"
# script_mode = "simulation"  # or "execution"

lh_entry = liquid_handler_config.get(liquid_handler_choice)
if lh_entry is None:
    raise ValueError(f"Unknown liquid handler: {liquid_handler_choice}")

# === Deck loading ===
deck_entry = lh_entry["deck"]

if isinstance(deck_entry, tuple):
    deck_module, deck_class = deck_entry
    deck_args = {}
elif isinstance(deck_entry, dict):
    deck_module = deck_entry["module"]
    deck_class = deck_entry["class"]
    deck_args = deck_entry.get("args", {})
else:
    raise ValueError(f"Invalid deck entry format: {deck_entry}")

exec(f"from {deck_module} import {deck_class} as Deck")
deck = Deck(**deck_args)

# === Backend loading ===
backend_entry = lh_entry.get(script_mode)
if backend_entry is None:
    raise ValueError(f"No backend configured for mode '{script_mode}' in '{liquid_handler_choice}'")

if isinstance(backend_entry, tuple):
    backend_module, backend_class = backend_entry
    backend_args = {}
elif isinstance(backend_entry, dict):
    backend_module = backend_entry["module"]
    backend_class = backend_entry["class"]
    backend_args = backend_entry.get("args", {})
else:
    raise ValueError(f"Invalid backend entry format: {backend_entry}")

exec(f"from {backend_module} import {backend_class} as Backend")
backend = Backend(**backend_args)

# === Create LiquidHandler ===
lh = LiquidHandler(backend=backend, deck=deck)

# === Logging setup ===
logger = logging.getLogger("pylabrobot")
logger.setLevel(logging.DEBUG)


In [4]:
lh = LiquidHandler(backend=backend, deck=deck)

await lh.setup()
vis = Visualizer(resource=lh)
await vis.setup()
if script_mode == 'simulation':
    from pylabrobot.resources import set_tip_tracking, set_volume_tracking
    set_tip_tracking(True), set_volume_tracking(False);
else:
    await lh.backend.disable_cover_control()
    await lh.backend.move_all_channels_in_z_safety()
    lh.backend.allow_firmware_planning = True # very powerful
    lh.backend.read_timeout = 240 # give your commands more time

Setting up the liquid handler.
Resource deck was assigned to the liquid handler.
Resource trash was assigned to the liquid handler.
Resource trash_core96 was assigned to the liquid handler.
Resource teaching_carrier was assigned to the liquid handler.
Websocket server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


### Function Defintions

In [5]:
from typing import List, Any, Generator

def divide_list_into_chunks(
        list_l: List[Any],
        chunk_size: int
    ) -> Generator[List[Any], None, None]:
    """
    Divides a list into smaller chunks of a specified size.

    Parameters:
    - list_l (List[Any]): The list to be divided into chunks.
    - chunk_size (int): The size of each chunk.

    Returns:
    - Generator[List[Any], None, None]: A generator that yields chunks of the list.
    """
    for i in range(0, len(list_l), chunk_size):
        yield list_l[i:i + chunk_size]

## Deck Setup

In [6]:
if any([liquid_handler_choice == "star", liquid_handler_choice == "starlet"]):

    from pylabrobot.resources import (
        TIP_CAR_480_A00, HTF, STF, TIP_50ul
    )
    
    tip_carrier = TIP_CAR_480_A00(name="tip carrier")
    
    tip_carrier[2] = tip_rack_1000ul_3 = HTF(name="tip_rack_1000ul_3", with_tips=False)
    tip_carrier[1] = tip_rack_1000ul_2 = HTF(name="tip_rack_1000ul_2")
    tip_carrier[0] = tip_rack_1000ul_1 = HTF(name="tip_rack_1000ul_1")
    
    lh.deck.assign_child_resource(tip_carrier, rails=15)
    
    tip_carrier_2 = TIP_CAR_480_A00(name="tip carrier 2")
    
    tip_carrier_2[2] = tip_rack_50ul_3 = TIP_50ul(name="tip_rack_50ul_3", with_tips=False)
    tip_carrier_2[1] = tip_rack_50ul_2 = TIP_50ul(name="tip_rack_50ul_2")
    tip_carrier_2[0] = tip_rack_50ul_1 = TIP_50ul(name="tip_rack_50ul_1")
    
    lh.deck.assign_child_resource(tip_carrier_2, rails=22)

    # Move 50ul tips in random source location to empty tip_rack, filled "down_left"
    dest_tip_spot_chunked = tip_rack_50ul_3.traverse(
            batch_size=lh.backend.num_channels,
            direction="down_left"
        ) 
    
    dest_tip_spot_iterator = iter(
        [tip_spot for column in dest_tip_spot_chunked for tip_spot in column]
    )
    
    n = 60
    random_source_numbers = random.sample(range(96), k=n)
    
    source_tip_spot_chunked =  divide_list_into_chunks(
        list_l=[tip_rack_50ul_1.children[idx] for idx in random_source_numbers],
        chunk_size = lh.backend.num_channels
    )
    
    for source_tip_spots in source_tip_spot_chunked:
    
        destination_tip_spots = [next(dest_tip_spot_iterator) for idx in source_tip_spots]
    
        await lh.pick_up_tips(
            source_tip_spots,
        )
    
        if script_mode == "simulation":
            time.sleep(1)
    
        await lh.drop_tips(
            destination_tip_spots,
            )

Resource tip carrier was assigned to the liquid handler.
Resource tip carrier was assigned to the liquid handler.
Resource tip carrier 2 was assigned to the liquid handler.
Resource tip carrier 2 was assigned to the liquid handler.
Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tip_rack_50ul_1_tipspot_7_3 0,0,0            HamiltonTip  65               8                    50.4             No        
  p1: tip_rack_50ul_1_tipspot_10_7 0,0,0            HamiltonTip  65               8                    50.4             No        
  p2: tip_rack_50ul_1_tipspot_1_2 0,0,0            HamiltonTip  65               8                    50.4             No        
  p3: tip_rack_50ul_1_tipspot_0_2 0,0,0            HamiltonTip  65               8                    50.4             No        
  p4: tip_rack_50ul_1_tipspot_0_4 0,0,0            HamiltonTip  65               8                    50.4 

In [7]:
if liquid_handler_choice == "ot2" :

    from pylabrobot.resources import (
        opentrons_96_filtertiprack_1000ul,
        opentrons_96_filtertiprack_20ul
    )

    tip_rack_1000ul_3 = opentrons_96_filtertiprack_1000ul(name="tip_rack_1000ul_3", with_tips=False)
    tip_rack_1000ul_2 = opentrons_96_filtertiprack_1000ul(name="tip_rack_1000ul_2")
    tip_rack_1000ul_1 = opentrons_96_filtertiprack_1000ul(name="tip_rack_1000ul_1")

    lh.deck.assign_child_at_slot(tip_rack_1000ul_3, slot=8)
    lh.deck.assign_child_at_slot(tip_rack_1000ul_2, slot=5)
    lh.deck.assign_child_at_slot(tip_rack_1000ul_1, slot=2)

    tip_rack_20ul_3 = opentrons_96_filtertiprack_20ul(name="tip_rack_20ul_3", with_tips=False)
    tip_rack_20ul_2 = opentrons_96_filtertiprack_20ul(name="tip_rack_20ul_2")
    tip_rack_20ul_1 = opentrons_96_filtertiprack_20ul(name="tip_rack_20ul_1")

    lh.deck.assign_child_at_slot(tip_rack_20ul_3, slot=9)
    lh.deck.assign_child_at_slot(tip_rack_20ul_2, slot=6)
    lh.deck.assign_child_at_slot(tip_rack_20ul_1, slot=3)

    # Move 20ul tips in random source location to empty tip_rack, filled "down_left"
    dest_tip_spot_chunked = tip_rack_20ul_3.traverse(
            batch_size=lh.backend.num_channels,
            direction="down_left"
        ) 
    
    dest_tip_spot_iterator = iter(
        [tip_spot for column in dest_tip_spot_chunked for tip_spot in column]
    )
    
    n = 60
    random_source_numbers = random.sample(range(96), k=n)
    
    source_tip_spot_chunked =  divide_list_into_chunks(
        list_l=[tip_rack_20ul_1.children[idx] for idx in random_source_numbers],
        chunk_size = lh.backend.num_channels
    )
    
    for source_tip_spots in source_tip_spot_chunked:
    
        destination_tip_spots = [next(dest_tip_spot_iterator) for idx in source_tip_spots]
    
        await lh.pick_up_tips(
            source_tip_spots,
        )
    
        if script_mode == "simulation":
            time.sleep(1)
    
        await lh.drop_tips(
            destination_tip_spots,
            )

In [8]:
if liquid_handler_choice == "evo150" :

    from pylabrobot.resources import (
        opentrons_96_filtertiprack_1000ul,
        opentrons_96_filtertiprack_20ul
    )

    # TODO: generate EVO150 showcase

In [9]:
dest_tip_spot_chunked = tip_rack_1000ul_3.traverse(
        batch_size=lh.backend.num_channels,
        direction="down_left"
    )

dest_tip_spot_iterator = iter(
    [tip_spot for column in dest_tip_spot_chunked for tip_spot in column]
)

n = 40
random_source_numbers = random.sample(range(96), k=n)

source_tip_spot_chunked =  divide_list_into_chunks(
    list_l=[tip_rack_1000ul_1.children[idx] for idx in random_source_numbers],
    chunk_size = lh.backend.num_channels
)

for source_tip_spots in source_tip_spot_chunked:

    destination_tip_spots = [next(dest_tip_spot_iterator) for idx in source_tip_spots]

    await lh.pick_up_tips(
        source_tip_spots,
    )

    if script_mode == "simulation":
        time.sleep(0.5)

    await lh.drop_tips(
        destination_tip_spots,
        )

# if script_mode == "simulation":
#     _ = [tip_rack_1000ul_1.children[idx].tracker.remove_tip() for idx in numbers]
#     _ = [tip_rack_1000ul_1.children[idx].tracker.commit() for idx in numbers]

Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tip_rack_1000ul_1_tipspot_0_6 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p1: tip_rack_1000ul_1_tipspot_3_1 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p2: tip_rack_1000ul_1_tipspot_3_3 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p3: tip_rack_1000ul_1_tipspot_2_2 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p4: tip_rack_1000ul_1_tipspot_10_3 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p5: tip_rack_1000ul_1_tipspot_8_4 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p6: tip_rack_1000ul_1_tipspot_9_6 0,0,0            HamiltonTip  1

In [10]:
n = 30
random_source_numbers = random.sample(range(96), k=n)

source_tip_spot_chunked =  divide_list_into_chunks(
    list_l=[tip_rack_1000ul_2.children[idx] for idx in random_source_numbers],
    chunk_size = lh.backend.num_channels
)

for source_tip_spots in source_tip_spot_chunked:

    destination_tip_spots = [next(dest_tip_spot_iterator) for idx in source_tip_spots]

    await lh.pick_up_tips(
        source_tip_spots,
    )

    if script_mode == "simulation":
        time.sleep(0.5)

    await lh.drop_tips(
        destination_tip_spots,
        )

# if script_mode == "simulation":
#     _ = [tip_rack_1000ul_1.children[idx].tracker.remove_tip() for idx in numbers]
#     _ = [tip_rack_1000ul_1.children[idx].tracker.commit() for idx in numbers]

Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tip_rack_1000ul_2_tipspot_8_7 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p1: tip_rack_1000ul_2_tipspot_6_7 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p2: tip_rack_1000ul_2_tipspot_5_5 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p3: tip_rack_1000ul_2_tipspot_1_5 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p4: tip_rack_1000ul_2_tipspot_4_1 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p5: tip_rack_1000ul_2_tipspot_3_3 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p6: tip_rack_1000ul_2_tipspot_3_6 0,0,0            HamiltonTip  10

In [11]:
from typing import List, Any, Generator
from pylabrobot.resources import TipRack


async def consolidate_tip_inventory(
    lh,
    ignore_tiprack_list: List[str] = ["teaching_tip_rack"]
    ):
    """
    Consolidate partial tip racks on the deck by redistributing tips.

    This function identifies partially-filled tip racks (excluding any in
    `ignore_tiprack_list`) and consolidates their tips into as few tip racks
    as possible, grouped by tip model. Tips are moved efficiently to minimize
    pipetting steps, avoiding redundant visits to the same drop columns.

    Args:
        lh: The liquid handler instance providing access to deck resources and
            pick/drop operations.
        ignore_tiprack_list: List of tip rack names to exclude from consolidation.

    Returns:
        None. The function performs in-place tip redistribution via async pick/drop.
    """
    
    def merge_sublists(
        lists: List[List[int]],
        max_len: int
        ) -> List[List[int]]:
        """
        Merge adjacent sublists if combined length <= max_len,
          without splitting sublists."""
        merged, buffer = [], []
    
        for sublist in lists:
            if not sublist:
                continue  # skip empty sublists
    
            if len(buffer) + len(sublist) <= max_len:
                buffer.extend(sublist)
            else:
                if buffer:
                    merged.append(buffer)
                buffer = sublist  # start new buffer
    
        if buffer:
            merged.append(buffer)
    
        return merged

    def divide_list_into_chunks(
            list_l: List[Any],
            chunk_size: int
        ) -> Generator[List[Any], None, None]:
        """
        Divides a list into smaller chunks of a specified size.
    
        Parameters:
        - list_l (List[Any]): The list to be divided into chunks.
        - chunk_size (int): The size of each chunk.
    
        Returns:
        - Generator[List[Any], None, None]: A generator that yields chunks of the list.
        """
        for i in range(0, len(list_l), chunk_size):
            yield list_l[i:i + chunk_size]
    
    all_tipracks_on_deck_list = [
        item for item in lh.get_all_children()
        if isinstance(item, TipRack) and item.name not in ignore_tiprack_list
    ]
    
    clusters_by_model = {}
    
    for idx, tip_rack in enumerate(all_tipracks_on_deck_list):
    
        # Only consider partially-filled tip_racks
        tip_status = [
                tip_spot.tracker.has_tip
                for tip_spot in tip_rack.children
            ]
        partially_filled = any(tip_status) and not all(tip_status)
    
        if partially_filled:
    
            tipspots_w_tips = [i for b, i in zip(tip_status, tip_rack.children) if b]
    
            # Identify model by hashed unique physical characteristics
            current_model = hash(tipspots_w_tips[0].tracker.get_tip())
            
            num_empty_tipspots = len(tip_status) - len(tipspots_w_tips)
            
            sanity_check = all(
                hash(tip_spot.tracker.get_tip())==current_model 
                for tip_spot in tipspots_w_tips[1:]
            )
            
            if sanity_check:
                clusters_by_model.setdefault(current_model, []).append((tip_rack, num_empty_tipspots))
    
    # Sort partially-filled tipracks by minimal fill_len
    for model, rack_list in clusters_by_model.items():
        rack_list.sort(key=lambda x: x[1])
    
    # Consolidate one tip model at a time across all tip_racks of that model
    for model, rack_list in clusters_by_model.items():

        print(f"Consolidating:\n - {', ' .join([rack.name for rack, num in rack_list])}")
   
        all_tip_spots_list = [tip for tip_rack, _ in rack_list for tip in tip_rack.children]
        
        # 1: Record current tip state
        current_tip_presence_list = [tip_spot.has_tip() for tip_spot in all_tip_spots_list]
    
        # 2: Generate target/consolidated tip state
        total_length = len(all_tip_spots_list)
        num_tips_per_model = sum(current_tip_presence_list)
    
        target_tip_presence_list = [
            # True if i < num_tips_per_model else False for i in range(total_length)
            i < num_tips_per_model for i in range(total_length)
        ]
    
        # 3: Calculate tip_spots involved in tip movement
        tip_movement_list = [
            c - t for c, t in zip(current_tip_presence_list, target_tip_presence_list)
        ]
    
        tip_origin_indices = [i for i, v in enumerate(tip_movement_list) if v == 1]
        all_origin_tip_spots = [all_tip_spots_list[idx] for idx in tip_origin_indices]
    
        tip_target_indices = [i for i, v in enumerate(tip_movement_list) if v == -1]
        all_target_tip_spots = [all_tip_spots_list[idx] for idx in tip_target_indices]
    
        # 4: Cluster target tip_spots by BOTH parent tip_rack & x-coordinate
        sorted_tip_spots = sorted(
            all_target_tip_spots,
            key=lambda tip: (str(tip.parent), round(tip.location.x, 3))
        )
        
        target_tip_clusters_by_parent_x = {}
        for tip_spot in sorted_tip_spots:
            key = (str(tip_spot.parent), round(tip_spot.location.x, 3))
            if key not in target_tip_clusters_by_parent_x :
                target_tip_clusters_by_parent_x [key] = []
            target_tip_clusters_by_parent_x[key].append(tip_spot)


        # Only continue if tip_racks are not already consolidated
        if len(target_tip_clusters_by_parent_x) > 0:

            current_tip_model = all_origin_tip_spots[0].tracker.get_tip()

            # Ensure there are channels that can pick up the tip model
            num_channels_available = len([
                c for c in range(lh.backend.num_channels)
                if lh.backend.can_pick_up_tip(c, current_tip_model)
            ])

            # 5: Optimise speed 
            if num_channels_available > 0:

                # by aggregating drop columns i.e. same drop column should not be visited twice!
                if num_channels_available >= 8: # physical constraint of tip_rack's having 8 rows
                    
                    merged_target_tip_clusters = merge_sublists(
                        target_tip_clusters_by_parent_x.values(),
                        max_len = 8
                    )

                else: # by chunking drop columns into size of available channels
                    
                    merged_target_tip_clusters = list(divide_list_into_chunks(
                        all_target_tip_spots,
                        chunk_size = num_channels_available
                    ))
                    
                len_transfers = len(merged_target_tip_clusters)
                
                # 6: Execute tip movement/consolidation
                for idx, target_tip_spots in enumerate(merged_target_tip_clusters):
                    print(f"     - tip transfer cycle: {idx} / {len_transfers-1}")
                    num_channels = len(target_tip_spots)
                    print(len(target_tip_spots))
                    use_channels = list(range(num_channels))
    
                    origin_tip_spots = [all_origin_tip_spots.pop(0) for idx in range(num_channels)]
    
                    await lh.pick_up_tips(
                        origin_tip_spots,
                        use_channels=use_channels
                    )
        
                    await lh.drop_tips(
                        target_tip_spots,
                        use_channels=use_channels
                    )
            else:
                print(f"Tips already optimally consolidated!")

        else:
            raise ValueError(f"No channel capable of handling tips on deck: {current_tip_model}")

In [12]:
lh.backend.num_channels

8

In [13]:
await consolidate_tip_inventory(
    lh=lh,
    ignore_tiprack_list = ["teaching_tip_rack", "tip_rack_1000ul_3"]
)

Consolidating:
 - tip_rack_1000ul_2, tip_rack_1000ul_1
     - tip transfer cycle: 0 / 5
4
Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tip_rack_1000ul_1_tipspot_3_5 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p1: tip_rack_1000ul_1_tipspot_3_6 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p2: tip_rack_1000ul_1_tipspot_3_7 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p3: tip_rack_1000ul_1_tipspot_4_0 0,0,0            HamiltonTip  1065             8                    95.1             Yes       
Dropping tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tip_rack_1000ul_1_tipspot_0_1 0,0,0            HamiltonTip  1065             8                    

In [14]:
import random
import matplotlib.pyplot as plt

# Parameters
total_length = 182
num_tips = 106

# Generate current tip list
random.seed(42)
current_tips_list = all_tip_presence_list

# Generate target tip list
target_tips_list = target_tips_list

# Compute movement list
tip_movement_list = [
    c - t for c, t in zip(current_tips_list, target_tips_list)
]

# Stack lists for visualization
tips_matrix = [current_tips_list, target_tips_list, tip_movement_list]

# Convert to 2D list of floats for imshow
tips_matrix_float = [[float(x) for x in row] for row in tips_matrix]

# Plot using matplotlib
fig, ax = plt.subplots(figsize=(12, 3))
cax = ax.imshow(tips_matrix_float, cmap="bwr", aspect="auto", vmin=-1, vmax=1)
ax.set_yticks([0, 1, 2])
ax.set_yticklabels(["Current", "Target", "Movement"])
ax.set_xticks([])
ax.set_title("Tip Movement Overview (No NumPy)")
plt.colorbar(cax, orientation='vertical', label='Tip Delta')
plt.tight_layout()
plt.show()


NameError: name 'all_tip_presence_list' is not defined